In [11]:
import chess
import chess.engine
import chess.pgn
import time
import collections
import random
import requests
import qrcode
import qrcode.image.svg
import json
import math

from numpy import interp
from IPython.display import display, SVG, HTML, clear_output

import_game_url = "https://lichess.org/api/import"

engine = chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish")
limit = chess.engine.Limit(time=0.1)

emoji_list = [
                "&#128561",
                "&#128552",
                "&#128546",
                "&#128534",
                "&#128533",
                "&#128528",
                "&#128578",
                "&#128512", 
                "&#128513",
                "&#128514",
                "&#128526",
            ]

def get_emoji_from_valuation(valuation):
  mate_score = 50000
  value = valuation.score(mate_score=mate_score)

  if value > 2000:
    return emoji_list[0]
  if value > 1000:
    return emoji_list[1]
  if value > 500:
    return emoji_list[2]
  if value > 200:
    return emoji_list[3]
  if value > 99:
    return emoji_list[5]
  elif value < -2000:
    return emoji_list[-1]
  elif value < -1000:
    return emoji_list[-2]
  elif value < -500:
    return emoji_list[-3]
  elif value < -200:
    return emoji_list[-4]
  elif value < -99:
    return emoji_list[-5]
  else:
    return emoji_list[5]


def random_player(board):
  move = random.choice(list(board.legal_moves))
  return move.uci()

def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

def human_player(board):
    display(board)
    uci = get_move("%s's move [q to quit]> " % who(board.turn))
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print("Legal moves: " + (",".join(sorted(legal_uci_moves))))
        uci = get_move("%s's move[q to quit]> " % who(board.turn))
    return uci

def who(player):
  return "White" if player == chess.WHITE else "Black"

def display_board(board, use_svg):
    if use_svg:
      return board._repr_svg_()
    else:
      return "<pre>" + str(board) + "</pre>"

def get_valuation(board):
  return engine.analyse(board, limit)

def create_pgn_from_board(board):
  game = chess.pgn.Game()

  # Undo all moves
  switchyard = collections.deque()
  while board.move_stack:
    switchyard.append(board.pop())
  
  game.setup(board)
  node = game

  # Replay all moves
  while switchyard:
    move = switchyard.pop()
    node = node.add_variation(move)
    board.push(move)
  
  game.headers["Result"] = board.result()
  return game

def play_game(player1, player2, visual="svg", pause=0.1):
  use_svg = (visual == "svg")
  board = chess.Board()

  try:
    while not board.is_game_over(claim_draw=True):
      if board.turn == chess.WHITE:
        uci = player1(board)
      else:
        uci = player2(board)
      name = who(board.turn)
      raw_valuation = get_valuation(board)
      valuation = get_valuation(board)['score'].pov(chess.WHITE)
      board.push_uci(uci)
      board_stop = display_board(board, use_svg)
      
      black_status = get_emoji_from_valuation(raw_valuation['score'].pov(chess.WHITE))
      white_status = get_emoji_from_valuation(raw_valuation['score'].pov(chess.BLACK))

      html = '<b>Move: %s %s</br>Play %s</br>Evaluation: %s</br> <p style="font-size:60px;margin:-4px 0 -4px 0px">%s</p> </br>%s</b><br/><p style="font-size:60px;margin:-4px 0 -4px 0px">%s</p>' % (
        len(board.move_stack), name, uci, valuation, black_status, board_stop, white_status
      )
      if visual is not None:
        if visual == "svg":
          clear_output(wait=True)
        display(HTML(html))
        if visual == "svg":
          time.sleep(pause)
  except KeyboardInterrupt:
    msg = "Game interrupted!"
    return (None, msg, board)
  result = None
  if board.is_checkmate():
    msg = "Checkmate: " + who(not board.turn) + " win!"
    result = not board.turn
  elif board.is_stalemate():
        msg = "Draw: stalemate"
  elif board.is_fivefold_repetition():
      msg = "Draw: 5-fold repetition"
  elif board.is_insufficient_material():
      msg = "Draw: insufficient material"
  elif board.can_claim_draw():
      msg = "Draw: claim"
  if visual is not None:
      print(msg)

  pgn = create_pgn_from_board(board)

  lichess_link = ""
  lichess_content_json = ""
  try:
    data = {'pgn': pgn}
    r = requests.post(import_game_url, data=data)
    lichess_content_json = r.content.decode("utf-8")
  except:
    print("Failed to import game to lichess")
    

  json_obj = json.loads(lichess_content_json)
  lichess_link = json_obj['url']
  print(lichess_link)

  qr_code = qrcode.make(lichess_link, image_factory=qrcode.image.svg.SvgImage)
  qr_code.save("generated/qrcode.svg")
  display(SVG("generated/qrcode.svg"))

  return (result, msg, board)

def staticAnalysis(board, move, my_color):
    score = random.random()
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 3), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 9), 
                           (chess.KNIGHT, 3),
                           (chess.ROOK, 5)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    score += 100 if board.is_checkmate() else 0
    return score

def player1(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

def engine_player(board):
  result = engine.play(board, limit)
  return result.move.uci()

play_game(player1, engine_player)

Checkmate: Black win!
https://lichess.org/hJ9mkG84


(False,
 'Checkmate: Black win!',
 Board('r3k2r/p1pp1p1p/p7/8/8/P1b1P3/2q2PbP/4K1NR w kq - 0 15'))

In [12]:
import requests, json

url = "https://www.lichess.org/api/games/user/marrespratt"

r = requests.get(url, params={"max":2, "analysed":"true", "clocks":"true", "evals":"true", "opening":"true"})
r_text = r.content.decode("utf-8")
print(r_text)
#data = json.loads(r_text)

[Event "Rated Blitz game"]
[Site "https://lichess.org/OHrEqFV8"]
[Date "2022.04.13"]
[White "aufdiebergen"]
[Black "Marrespratt"]
[Result "0-1"]
[UTCDate "2022.04.13"]
[UTCTime "16:42:01"]
[WhiteElo "1448"]
[BlackElo "1447"]
[WhiteRatingDiff "-5"]
[BlackRatingDiff "+39"]
[Variant "Standard"]
[TimeControl "180+0"]
[ECO "B10"]
[Opening "Caro-Kann Defense: Breyer Variation"]
[Termination "Time forfeit"]

1. e4 { [%eval 0.33] [%clk 0:03:00] } 1... c6 { [%eval 0.56] [%clk 0:03:00] } 2. d3 { [%eval 0.14] [%clk 0:02:59] } 2... d5 { [%eval 0.32] [%clk 0:02:59] } 3. Qf3 { [%eval -0.53] [%clk 0:02:55] } 3... Nf6 { [%eval 0.28] [%clk 0:02:51] } 4. e5 { [%eval 0.28] [%clk 0:02:48] } 4... Nfd7 { [%eval 0.39] [%clk 0:02:50] } 5. d4 { [%eval -0.19] [%clk 0:02:46] } 5... c5 { [%eval 0.43] [%clk 0:02:49] } 6. c3 { [%eval -0.04] [%clk 0:02:45] } 6... Nc6 { [%eval 0.16] [%clk 0:02:44] } 7. Nh3 { [%eval -3.15] [%clk 0:02:34] } 7... cxd4 { [%eval -3.11] [%clk 0:02:42] } 8. Ng5 { [%eval -5.32] [%clk 0:02:28

In [13]:
import requests

endpoint = "https://lichess.org/api/import"
            
pgn = """
[Event "Casual Blitz game"]
[Site "https://lichess.org/tvdw3N53"]
[Date "2022.04.30"]
[White "lichess AI level 1"]
[Black "Marrespratt"]
[Result "0-1"]
[UTCDate "2022.04.30"]
[UTCTime "09:28:22"]
[WhiteElo "?"]
[BlackElo "1488"]
[Variant "Standard"]
[TimeControl "180+2"]
[ECO "B10"]
[Termination "Normal"]

1. e4 c6 2. Nf3 d5 3. exd5 cxd5 4. d4 Nc6 5. Bd3 Bg4 6. c3 e6 7. h3 Bxf3 8. Qxf3 Nf6 9. Nd2 Bd6 10. O-O O-O 11. Qd1 a6 12. Nb3 b5 13. Re1 e5 14. dxe5 Nxe5 15. Bf5 Nc4 16. Nd4 Re8 17. Bc2 Rxe1+ 18. Qxe1 Qc7 19. Nf5 Bh2+ 20. Kf1 Re8 21. Be3 Nxb2 22. Qe2 Nc4 23. Re1 a5 24. Rb1 b4 25. g3 bxc3 26. Qd3 Nd2+ 27. Ke2 Nxb1 28. Nd4 Qc4 29. Kf3 Qxd3 30. Bxb1 Bxg3 31. Bc2 Rxe3+ 32. Kg2 Qxd4 33. Be4 c2 34. Bg6 hxg6 35. h4 c1=Q 36. f3 Qc2+ 37. Kh3 Qxh4# 0-1
"""

data = {'pgn': pgn}

r = requests.post(endpoint, data=data)
r_text = r.content.decode("utf-8")
print(r)
print(r_text)

<Response [200]>
{"id":"PLqa8uS8","url":"https://lichess.org/PLqa8uS8"}


In [14]:
import requests
from requests.auth import HTTPBasicAuth

auth_token = 'access_token lip_UzqKpePwZw1shYm3cYEo'

url = 'https://lichess.org/api/games/export/_ids'

data = "TJxUmbWK,4OtIh2oh,ILwozzRZ"

r = requests.post(url, data=data) 
print(r)
print(r.content.decode("utf-8"))

<Response [200]>
[Event "Daily Classical Arena"]
[Site "https://lichess.org/TJxUmbWK"]
[Date "2017.08.30"]
[White "arex"]
[Black "JERC-12Jesus"]
[Result "1-0"]
[UTCDate "2017.08.30"]
[UTCTime "20:40:27"]
[WhiteElo "1627"]
[BlackElo "1740"]
[WhiteRatingDiff "+15"]
[BlackRatingDiff "-15"]
[Variant "Standard"]
[TimeControl "600+0"]
[ECO "B07"]
[Termination "Normal"]

1. e4 d6 2. d4 Nf6 3. Nc3 g6 4. Bd3 Bg7 5. Be3 O-O 6. Qd2 Ng4 7. O-O-O Nxe3 8. Qxe3 Nd7 9. f4 Nf6 10. h3 c6 11. g4 Qb6 12. Nf3 c5 13. dxc5 Qxc5 14. Qxc5 dxc5 15. e5 Nd7 16. Nd5 e6 17. Ne7+ Kh8 18. Bb5 a6 19. Bxd7 Bxd7 20. Rxd7 Rfe8 21. Rxb7 1-0


[Event "Daily Classical Arena"]
[Site "https://lichess.org/4OtIh2oh"]
[Date "2017.08.30"]
[White "wowk"]
[Black "arex"]
[Result "1-0"]
[UTCDate "2017.08.30"]
[UTCTime "20:21:03"]
[WhiteElo "1662"]
[BlackElo "1638"]
[WhiteRatingDiff "+10"]
[BlackRatingDiff "-11"]
[Variant "Standard"]
[TimeControl "600+0"]
[ECO "C89"]
[Termination "Time forfeit"]

1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 N

In [15]:
from IPython.display import display, SVG, HTML, clear_output
from numpy import interp
import emoji

emoji_list = [
                "&#128561",
                "&#128552",
                "&#128546",
                "&#128534",
                "&#128533",
                "&#128528",
                "&#128578",
                "&#128512", 
                "&#128513",
                "&#128514",
                "&#128526",
            ]

for e in emoji_list:
    html = '<p style="font-size:60px">%s</p>' % (e)
    display(HTML(html))

